# 제 2회 기출문제

## 문제 1. 
주어진 데이터에서 'views' 컬럼을 기준으로 내림차순 정렬하고,<br>
'views' 컬럼 상위 10개 데이터를 'views' 컬럼 상위 10번째 값으로 대체한 후 <br>
'age' 컬럼이 80 이상인 데이터의 'views' 컬럼 평균값을 구하시오.(반올림하여 소수 둘째 자리까지 계산)



In [9]:
import pandas as pd

df = pd.read_csv('./ch2_data/members.csv')
df

,id,age,city,f1,f2,f3,f4,f5,subscribed,views
0,id01,2.0,서울,NaN,0,gold,ENFJ,91.297791,2021-07-16,6820.0
1,id02,9.0,서울,70.0,1,NaN,ENFJ,60.339826,2021-05-12,2534.0
2,id03,27.0,서울,61.0,1,gold,ISTJ,17.252986,2021-03-16,7312.0
3,id04,75.0,서울,NaN,2,NaN,INFP,52.667078,2021-07-21,493.0
4,id05,24.0,서울,85.0,2,NaN,ISFJ,29.269869,2021-03-07,1338.0
...,...,...,...,...,...,...,...,...,...,...
95,id95,77.0,경기,43.0,1,gold,INTJ,91.297791,2021-05-21,8697.0
96,id96,92.0,경기,53.0,1,silver,ENTJ,52.667078,2021-05-06,4336.0
97,id97,100.0,경기,NaN,0,gold,INFP,67.886373,2021-03-18,6687.0
98,id98,39.0,경기,58.0,2,NaN,INFP,98.429899,2021-10-02,865.0


In [12]:
# 1. views 컬럼 내림차순 정렬

df.sort_values('views', ascending=False, inplace=True)

# 2. 상위 10개 중에서 10번째 값 최솟값 구하기
min_value = df['views'][:10].min()

# 3. 상위 10개 중에서 10번째 값(최솟값) 대체
df.iloc[:10,-1] = min_value

# 4. age가 80이상의 views 컬럼 평균
conf = df['age']>=80
result = df[conf]['views'].mean()
print(round(result, 2))

5674.04


## 문제 2. 
주어진 데이터셋에서 앞에서부터 순서대로 80% 데이터만 활용해 <br>
'f1' 컬럼 결측치를 중앙 값으로 채우기 전후의 표준편차를 구하고 두 표본 표준편차의 차이를 계산하시오<br>
(두 표본 표준편차는 절대값으로 계산하고, 반올림하여 소수 둘째 자리까지 계산)


In [13]:
import pandas as pd
df = pd.read_csv('./ch2_data/members.csv')

In [16]:
line = int(len(df)*0.8)

df = df.iloc[:line]

In [18]:
std1 = df['f1'].std()
med = df['f1'].median()
df['f1'] = df['f1'].fillna(med)
std2 = df['f1'].std()

result = abs(std1-std2)
print(round(result,2))

3.56


## 문제 3.
주어진 데이터셋 'age' 컬럼의 이상치를 모두 더하시오.<br>
이상치는 평균으로부터 '표준편차*1.5'를 벗어나는 영역을 이상치라고 판단


In [19]:
import pandas as pd
df = pd.read_csv('./ch2_data/members.csv')

In [24]:
std1 = df['age'].std()
mean = df['age'].mean()

lower = mean - (std1 * 1.5)
upper = mean + (std1 * 1.5)
cond1 = df['age']<lower
cond2 = df['age']>upper

print(df[cond1|cond2]['age'].sum())

473.5


## 문제 4.
제품 배송 시간에 맞춰 배송되지 않을 확률값을 예측하시오.
- 예측할 컬럼 : Reached.on.Time_Y.N(0: 정시 도착, 1: 정시 도착 x)
- 평가 기준 : ROC_AUC로 평가


In [26]:
import pandas as pd

X_test = pd.read_csv('./ch2_data/X_test.csv')
X_train = pd.read_csv('./ch2_data/X_train.csv')
y_train = pd.read_csv('./ch2_data/y_train.csv')


In [29]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)

(8799, 11)
(2200, 11)
(8799, 2)


In [32]:
X_train.isnull().sum()

ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
dtype: int64

In [33]:
X_test.isnull().sum()

ID                     0
Warehouse_block        0
Mode_of_Shipment       0
Customer_care_calls    0
Customer_rating        0
Cost_of_the_Product    0
Prior_purchases        0
Product_importance     0
Gender                 0
Discount_offered       0
Weight_in_gms          0
dtype: int64

In [34]:
y_train['Reached.on.Time_Y.N'].value_counts()

Reached.on.Time_Y.N
1    5236
0    3563
Name: count, dtype: int64

In [38]:
from sklearn.preprocessing import LabelEncoder

cols = X_train.select_dtypes(include='object').columns
 

for col in cols:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col])
    X_test[col] = le.transform(X_test[col])


In [39]:
X_train = X_train.drop('ID',axis=1)

In [40]:
X_test_id = X_test.pop('ID')

In [47]:
from sklearn.model_selection import train_test_split
X_tr,X_val,y_tr,y_val = train_test_split(
    X_train,
    y_train['Reached.on.Time_Y.N'],
    test_size=0.2,
    random_state=0
)


X_tr.shape,X_val.shape,y_tr.shape,y_val.shape

((7039, 10), (1760, 10), (7039,), (1760,))

In [44]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

In [49]:
# 로지스틱회귀
lr = LogisticRegression(random_state=0)
lr.fit(X_tr,y_tr)
pred = lr.predict_proba(X_val)
print(roc_auc_score(y_val, pred[:,1]))

0.7294862508383635


/Users/gihyun/Desktop/dev/GitHub/-BigDataAnalyst/.venv/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [50]:
dt = DecisionTreeClassifier(random_state=0)
dt.fit(X_tr,y_tr)
pred = dt.predict_proba(X_val)
print(roc_auc_score(y_val,pred[:,1]))

0.6540107310529846


In [51]:
rf = RandomForestClassifier(random_state=0)
rf.fit(X_tr,y_tr)
pred = rf.predict_proba(X_val)
print(roc_auc_score(y_val,pred[:,1]))

0.7657780013413816


In [53]:
pred = rf.predict_proba(X_test)
pred

array([[0.61, 0.39],
       [0.61, 0.39],
       [0.67, 0.33],
       ...,
       [0.58, 0.42],
       [0.66, 0.34],
       [0.4 , 0.6 ]], shape=(2200, 2))

In [54]:
submit = pd.DataFrame({
    'ID':X_test_id,
    'Reached.on.Time_Y.N':pred[:,1]
})

In [55]:
submit.head(5)

,ID,Reached.on.Time_Y.N
0,8285,0.39
1,10192,0.39
2,8675,0.33
3,5753,0.42
4,448,0.97


### 심화학습


In [70]:
import pandas as pd

X_test = pd.read_csv('./ch2_data/X_test.csv')
X_train = pd.read_csv('./ch2_data/X_train.csv')
y_train = pd.read_csv('./ch2_data/y_train.csv')


In [71]:
from sklearn.preprocessing import LabelEncoder
cols = X_train.select_dtypes(include='object').columns

for col in cols:
    le = LabelEncoder()
    X_train[col] = le.fit_transform(X_train[col])
    X_test[col] = le.transform(X_test[col])


In [72]:
X_train = X_train.drop('ID',axis=1)
X_test_id = X_test.pop('ID')

In [76]:
from sklearn.preprocessing import MinMaxScaler

cols=['Customer_care_calls','Customer_rating','Cost_of_the_Product',
      'Prior_purchases','Discount_offered','Weight_in_gms'
      ]

scaler = MinMaxScaler()
X_train[cols] = scaler.fit_transform(X_train[cols])
X_test[cols] = scaler.fit_transform(X_test[cols])


In [77]:
from sklearn.model_selection import train_test_split
X_tr,X_val,y_tr,y_val = train_test_split(
    X_train,
    y_train['Reached.on.Time_Y.N'],
    test_size=0.2,
    random_state=0
)


X_tr.shape,X_val.shape,y_tr.shape,y_val.shape

((7039, 10), (1760, 10), (7039,), (1760,))

In [78]:
rf = RandomForestClassifier(random_state=0)
rf.fit(X_tr,y_tr)
pred = rf.predict_proba(X_val)
print(roc_auc_score(y_val,pred[:,1]))

0.7657256874580818


# 제 3회 기출문제


## 1. 유형

### 문제1.
결측치가 있는 데이터의 행을 제거하고 앞에서부터 '70%' 데이터만 활용해<br>
'f1' 컬럼 1사분위 값을 구하시오.(소수점 제거 후 정수 출력)

In [90]:
import pandas as pd
df = pd.read_csv('./ch3_data/members.csv')

df = df.dropna()
lines = int(len(df)*0.7)
df = df.iloc[:lines]

print(int(df['f1'].quantile(.25)))

57


### 문제2.
2000년 데이터 중 2000년 평균보다 큰 값의 데이터 수를 구하시오
(index 연도)


In [91]:
import pandas as pd
df = pd.read_csv('./ch3_data/year.csv',index_col='Unnamed: 0')

In [ ]:
# 풀이 1. 행을 기준으로 평균 계산

m = df.loc[2000].mean()

print(sum(df.loc[2000,:]>m))

In [105]:
# 풀이 2. 행과 열을 변경해 평균 계산

df = df.T

m = df[2000].mean()
print(len(df[df[2000]>m]))

100


### 문제 3. 
결측치가 가장 많은 컬럼명을 출력하시오.

In [106]:
import pandas as pd
df = pd.read_csv("./ch3_data/members.csv")

In [109]:
df_cntNull = df.isnull().sum()

df_cntNull = df_cntNull.sort_values(ascending=False)
print(df_cntNull.index[0])

f1


### 문제 4. 
여행 보험 패키지 상품을 구매할 확률값을 예측하시오.<br>
예측할 컬럼 : Travellnsurance(패키지 구매 여부 0:구매 x, 1: 구매함) <br>
모델 성능은 roc-auc 평가지표

In [110]:
import pandas as pd
train = pd.read_csv('./ch3_data/train.csv')
test = pd.read_csv('./ch3_data/test.csv')

In [111]:
train.shape,test.shape

((1490, 10), (497, 9))

In [112]:
# train 결측치 x
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           1490 non-null   int64 
 1   Age                  1490 non-null   int64 
 2   Employment Type      1490 non-null   object
 3   GraduateOrNot        1490 non-null   object
 4   AnnualIncome         1490 non-null   int64 
 5   FamilyMembers        1490 non-null   int64 
 6   ChronicDiseases      1490 non-null   int64 
 7   FrequentFlyer        1490 non-null   object
 8   EverTravelledAbroad  1490 non-null   object
 9   TravelInsurance      1490 non-null   int64 
dtypes: int64(6), object(4)
memory usage: 116.5+ KB


In [114]:
# test 결측치 x
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 497 entries, 0 to 496
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           497 non-null    int64 
 1   Age                  497 non-null    int64 
 2   Employment Type      497 non-null    object
 3   GraduateOrNot        497 non-null    object
 4   AnnualIncome         497 non-null    int64 
 5   FamilyMembers        497 non-null    int64 
 6   ChronicDiseases      497 non-null    int64 
 7   FrequentFlyer        497 non-null    object
 8   EverTravelledAbroad  497 non-null    object
dtypes: int64(5), object(4)
memory usage: 35.1+ KB


In [115]:
train['TravelInsurance'].value_counts()

TravelInsurance
0    965
1    525
Name: count, dtype: int64

In [116]:
train.describe(include='object')

,Employment Type,GraduateOrNot,FrequentFlyer,EverTravelledAbroad
count,1490,1490,1490,1490
unique,2,2,2,2
top,Private Sector/Self Employed,Yes,No,No
freq,1056,1270,1175,1209


In [118]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           1490 non-null   int64 
 1   Age                  1490 non-null   int64 
 2   Employment Type      1490 non-null   object
 3   GraduateOrNot        1490 non-null   object
 4   AnnualIncome         1490 non-null   int64 
 5   FamilyMembers        1490 non-null   int64 
 6   ChronicDiseases      1490 non-null   int64 
 7   FrequentFlyer        1490 non-null   object
 8   EverTravelledAbroad  1490 non-null   object
 9   TravelInsurance      1490 non-null   int64 
dtypes: int64(6), object(4)
memory usage: 116.5+ KB


In [119]:
target = train.pop('TravelInsurance')

In [120]:
train = pd.get_dummies(train)
test = pd.get_dummies(test)

In [121]:
from sklearn.model_selection import train_test_split
X_tr,X_val,y_tr,y_val = train_test_split(
    train,
    target,
    test_size=0.2,
    random_state=0
)
X_tr.shape,X_val.shape,y_tr.shape,y_val.shape

((1192, 13), (298, 13), (1192,), (298,))

In [123]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

In [124]:
lr = LogisticRegression(random_state=0)

lr.fit(X_tr,y_tr)
pred = lr.predict_proba(X_val)
print(roc_auc_score(y_val,pred[:,1]))

0.7745126705653022


/Users/gihyun/Desktop/dev/GitHub/-BigDataAnalyst/.venv/lib/python3.13/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [126]:
dt = DecisionTreeClassifier(random_state=0)
dt.fit(X_tr,y_tr)
pred = dt.predict_proba(X_val)
print(roc_auc_score(y_val, pred[:,1]))

0.7090155945419103


In [127]:
rf = RandomForestClassifier(random_state=0)

rf.fit(X_tr,y_tr)
pred = rf.predict_proba(X_val)
print(roc_auc_score(y_val,pred[:,1]))

0.850682261208577


In [128]:
pred = rf.predict_proba(test)
pred

array([[0.76, 0.24],
       [0.93, 0.07],
       [0.89, 0.11],
       [0.02, 0.98],
       [0.72, 0.28],
       [0.  , 1.  ],
       [0.53, 0.47],
       [0.93, 0.07],
       [0.22, 0.78],
       [0.95, 0.05],
       [0.93, 0.07],
       [0.92, 0.08],
       [0.7 , 0.3 ],
       [0.93, 0.07],
       [0.09, 0.91],
       [0.86, 0.14],
       [0.58, 0.42],
       [0.85, 0.15],
       [0.6 , 0.4 ],
       [0.38, 0.62],
       [0.88, 0.12],
       [0.88, 0.12],
       [0.  , 1.  ],
       [0.04, 0.96],
       [1.  , 0.  ],
       [0.01, 0.99],
       [0.01, 0.99],
       [0.86, 0.14],
       [0.02, 0.98],
       [0.01, 0.99],
       [0.91, 0.09],
       [0.73, 0.27],
       [0.9 , 0.1 ],
       [0.68, 0.32],
       [0.88, 0.12],
       [0.13, 0.87],
       [0.87, 0.13],
       [0.82, 0.18],
       [0.04, 0.96],
       [0.79, 0.21],
       [0.91, 0.09],
       [1.  , 0.  ],
       [0.69, 0.31],
       [0.74, 0.26],
       [0.08, 0.92],
       [0.86, 0.14],
       [0.02, 0.98],
       [0.74,

In [131]:
submit = pd.DataFrame({
    'index':test.index,
    'y_pred':pred[:,1]})

In [133]:
# submit.to_csv('./ch3_data/result.csv',index=False)

#### 심화 
기존 점수 : 0.85
RandomForestClassifier

In [ ]:
import pandas as pd

train = pd.read_csv('./ch3_data/train.csv')
test = pd.read_csv('./ch3_data/test.csv')

In [135]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1490 entries, 0 to 1489
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   Unnamed: 0           1490 non-null   int64 
 1   Age                  1490 non-null   int64 
 2   Employment Type      1490 non-null   object
 3   GraduateOrNot        1490 non-null   object
 4   AnnualIncome         1490 non-null   int64 
 5   FamilyMembers        1490 non-null   int64 
 6   ChronicDiseases      1490 non-null   int64 
 7   FrequentFlyer        1490 non-null   object
 8   EverTravelledAbroad  1490 non-null   object
 9   TravelInsurance      1490 non-null   int64 
dtypes: int64(6), object(4)
memory usage: 116.5+ KB


In [136]:
target = train.pop('TravelInsurance')

In [138]:
from sklearn.preprocessing import LabelEncoder

cols = train.select_dtypes(include='O').columns
for col in cols:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])


In [140]:
from sklearn.model_selection import train_test_split

X_tr,X_val,y_tr,y_val = train_test_split(
    train,target,test_size=0.2, random_state=0
)
X_tr.shape,X_val.shape,y_tr.shape,y_val.shape

((1192, 9), (298, 9), (1192,), (298,))

In [142]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score

rf = RandomForestClassifier(random_state=0)

rf.fit(X_tr,y_tr)
pred = rf.predict_proba(X_val)
print(roc_auc_score(y_val,pred[:,1]))

0.8630847953216375
